# Practical session 8: exporatory data analysis with `pandas`

Wednesday 29 November 2023

In this practical session, we'll examine how to carry out some exploratory data analysis using the `pandas` library. At the same time, we'll combine this with visualization of the analyses, making use of the `seaborn` statistical visualization library.

First, we'll import both libraries, using the command below:

In [ ]:
import pandas as pd
import seaborn as sns

### Reading CSVs into DataFrame objects

The [pd.read_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) function is a flexible tool for reading CSV files. It can handle multiple formats for its input, and it will infer types for columns on import. Instead of file names, the function equally takes URLs as input, which is what we'll use in the command below. We'll make use of the same dataset that we used during the lecture on Wednesday: the [Palmer Penguins dataset](https://allisonhorst.github.io/palmerpenguins/).

In [ ]:
df = pd.read_csv(
    "http://ccl.kuleuven.be/Courses/scripting/practical8/penguins_raw.csv")

`df` is a `pd.DataFrame`. These are perfectly suited for tabular data - data you would typically put in a spreadsheet. They display nicely in notebooks (the `head` function shows the first 5 instances by default):

In [ ]:
df.head()

### Dataframe shape

We can use the `shape` attribute in order to inspect how many rows (data instances) and columns (features) we have in our dataset:


In [ ]:
df.shape

## Creating DataFrames in other ways

Note that you can create `DataFrame` objects in different ways, using the commands below. This can be useful if you have your data in Python lists or dictionaries, and you want to create a DataFrame out of your data for analysis.

In [ ]:
pd.DataFrame(
    {
        "weight": ["68", "78", "75"],
        "length": ["170", "178", "174"]
    })

In [ ]:
pd.DataFrame(
    [
        ["68", "170"],
        ["78", "178"],
        ["75", "174"]
    ],
    columns=["weight", "length"])

## Columns as Series

The columns in DataFrames are `pd.Series` objects. You can pull them out by indexing directly into the `pd.DataFrame` with the column name (as a `str`):

In [ ]:
df['Species']

## Creating a Series directly

It's also possible to directly construct a `Series` object using a Python `list`.

In [ ]:
pd.Series(["68", "78", "75"], name="weight")

## Common series methods

The commands below showcase a number of a couple of common `Series` methods, which should be self-explanatory.

In [ ]:
df['Body Mass (g)'].mean()

In [ ]:
df['Body Mass (g)'].max()

In [ ]:
df['Body Mass (g)'].min()

In [ ]:
df['Body Mass (g)'].median()

`describe` gives a nice summary, including [quartile figures](https://en.wikipedia.org/wiki/Quartile).

In [ ]:
df['Body Mass (g)'].describe()

## Selecting slices of rows/columns

There are two ways to extract slices of rows/columns from your `DataFrame`. You can either use `loc`, which is used to select rows/columns by label name; or you can use `iloc`, which selects by row/column index:

In [ ]:
df.loc[:,'Species':'Island']

And based on the index:

In [ ]:
df.iloc[:,2:5]

## Subframes based on specific row values

By creating a Boolean series object, it is possible to filter a `DataFrame` based on specific values.

### Creating Boolean Series

A Boolean series object, which checks whether a certain condition is true for a particular instance, can be created as follows:

In [ ]:
df['Island'] == 'Torgersen'

### Boolean series as filter

Such a Boolean filter can then be used in order to filter instances from the dataframe based on that condition:

In [ ]:
df.loc[df['Island'] == 'Torgersen']

We can equally combine conditions in order to perform a more complex filtering based on multiple features:

In [ ]:
df.loc[(df['Island'] == 'Torgersen') & (df['Body Mass (g)'] > 4500)]

## Sorting

Sorting all instances based on a single feature, from small to large:

In [ ]:
df.sort_values('Flipper Length (mm)')

And from large to small:

In [ ]:
df.sort_values('Flipper Length (mm)', ascending=False)

## Counting instances based on a categorical variable

For categorical variables, we can count how many instances have the same feature:

In [ ]:
df['Species'].value_counts()

And we can visualize these counts, e.g. using a horizontal bar plot:

In [ ]:
df['Species'].value_counts().plot.barh()

## Summarizing discrete data

It is possible to create groups of data based on the discrete values in specific columns, i.e. aggregate the data of individual items, in order to create summary statistics. As an example, we can group our penguin instances based on the sex of the penguin (male or female):

In [ ]:
df.groupby('Sex')

Note that the command does not return any data; it just returns an object that we can iterate over. We would typically combine the `groupby` method with an additional function application, which carries out the actual aggregation; e.g. `mean()`:

In [ ]:
df.groupby('Sex').mean()

Often we are interested in the aggregation statistic over a particular feature; e.g. what is the average flipper length of male vs. female penguins? This can be done through selection a specific column:

In [ ]:
df.groupby('Sex')['Flipper Length (mm)'].mean()

And we can equally create a bar plot from this data:

In [ ]:
df.groupby('Sex')['Flipper Length (mm)'].mean().plot.bar()

### Contingency table

If we want to examine the relationship between two categorical variables, we can perform a cross-tabulation, also known as contingency table. For example, examining the relationship between the sex of the penguin and the island where the penguins live:  

In [ ]:
pd.crosstab(df['Sex'], df['Island'])

Or, normalized to 1, and including the marginals (sums across the variables):

In [ ]:
pd.crosstab(df['Sex'], df['Island'], normalize=True, margins=True)

Visualization is equally possible:

In [ ]:
pd.crosstab(df['Sex'], df['Island'], normalize=True, margins=True).plot.barh()

### Boxplots

Boxplots are often used to gauge the spread of a dataset. As an example, we can compare the boxplots for the three penguin species in our dataset:

In [ ]:
sns.boxplot(x='Body Mass (g)', y='Species', data=df)

We can see that the median for the Adelie and Chinstrap species is the same - but the boxplot for the former species indicates that the values are a bit more spread out (i.e. there is a bit more variation in their weight).

### Histograms

A histogram allows us to get an idea of the frequency distribution of the data;

In [ ]:
sns.displot(x='Body Mass (g)', data=df)

We can compare that to the simple boxplot for the weight variable:

In [ ]:
sns.boxplot(x='Body Mass (g)', data=df)

It is equally possible to plot multiple histograms for different categorical variables - in this case species and sex:

In [ ]:
sns.displot(x='Body Mass (g)', col='Species', row='Sex', data=df)

### Scatter plot

Finally, let's look at some scatter plots, in order to inspect the relationship between two continuous variables. The following command creates a scatter plot, plotting flipper length against body mass:

In [ ]:
sns.relplot(x='Flipper Length (mm)', y='Body Mass (g)', data=df)

We see a straight line emerging, so we can suspect the variables are strongly correlated. Using the `corr` method, which (by default) computes a [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient), we can see that this is indeed the case.

In [ ]:
df['Flipper Length (mm)'].corr(df['Body Mass (g)'])

If we have several groups, we can use different colors (`hue`) to distinguish between them, e.g. the penguins living on different islands:

In [ ]:
sns.relplot(x='Flipper Length (mm)', y='Body Mass (g)',
hue='Island', data=df)

Finally, let's plot a linear regression model (flipper length vs culmen length) for three subsets of the data (i.e. the different penguin species):

In [ ]:
sns.lmplot(x='Flipper Length (mm)', y='Culmen Length (mm)',
hue='Species', markers=['o', '^', 's'], data=df)

### Exercises

#### Exercise 1

What is the mean flipper length (in mm) for female Adelie Penguins living on the Torgersen Island?

In [ ]:
# insert your code here
df.loc[(df['Species'] == 'Adelie Penguin (Pygoscelis adeliae)') & (df['Sex'] == 'FEMALE') & (df['Island'] == 'Torgersen'), 'Flipper Length (mm)'].mean()


#### Exercise 2

Visually compare the clutch completion numbers (i.e., did the penguin finish laying all of its eggs) for the three different islands (Torgersen, Dream, Biscoe). Is there an island that's lagging behind on clutch completion?

In [ ]:
# insert your code here
pd.crosstab(df['Island'], df['Clutch Completion']).plot.barh()

#### Exercise 3

Compute the correlation coefficient between body mass and culmen length, for both female and male Gentoo penguins (separately). Create a linear regression plot for the two subsets in a single plot.

In [ ]:
# insert your code here
df.loc[(df['Species'] == 'Gentoo penguin (Pygoscelis papua)') & (df['Sex'] == 'FEMALE'), 'Body Mass (g)'].corr(df['Culmen Length (mm)'])


In [ ]:
df.loc[(df['Species'] == 'Gentoo penguin (Pygoscelis papua)') & (df['Sex'] == 'MALE'), 'Body Mass (g)'].corr(df['Culmen Length (mm)'])

In [ ]:
sns.lmplot(x='Body Mass (g)', y='Culmen Length (mm)',
           hue='Sex', markers=['o', '^'],
           data=df.loc[df['Species'] == 'Gentoo penguin (Pygoscelis papua)'])